In [1]:
import okx.MarketData as MarketData
import tkinter as tk
from threading import Thread, Event
import time
import tkinter as tk
from PIL import ImageGrab, Image
import numpy as np


# 初始化API
flag = "0"  # 实盘:0 , 模拟盘：1
marketDataAPI = MarketData.MarketAPI(flag=flag)
instID = ['BTC', 'ETH']

# 创建主窗口
root = tk.Tk()
main_x = 120
main_y = 80
# root.geometry(f"{main_x}x{main_y}+{0-9}+{int(1448-main_y)}")
root.geometry(f"{main_x}x{main_y}+{100}+{int(1000-main_y)}")
root.attributes('-topmost', True)
root.attributes('-transparentcolor', 'white')
root.config(bg='white')
root.overrideredirect(True)

label = tk.Label(root, text="", font=("Helvetica", 9), bg='white', anchor='w', justify='left')
label.pack(pady=1, padx=1, fill='both', expand=True)

# 全局变量
display_text = ""
heart = "0"  # 状态位初始为 "0"
is_dragging = False  # 是否拖动的标志
start_x = 0
start_y = 0

# 矩形区域的设置
rect_win = None  # 矩形窗口的引用
canvas =  None  
# 线程停止事件
stop_event = Event()


def get_average_color(image):
    """计算图像的平均颜色"""
    np_image = np.array(image)
    # 确保图像数据有效
    if np_image.size == 0:
        return (0, 0, 0)
    
    # 计算每个像素的平均值（RGB）
    avg_color = np_image[:, :, :3].mean(axis=(0, 1))
    # print(avg_color)
    return tuple(map(int, avg_color))

def update_label_background():
    global label, rect_win

    # 获取矩形窗口的区域
    x = root.winfo_rootx()
    y = root.winfo_rooty()
    width = root.winfo_width()
    height = root.winfo_height()


    # image = ImageGrab.grab(bbox=(x, y, x + width, y + height))
    # 扩大截图区域5%
    expand_ratio = 0.2
    expanded_width = width * (1 + expand_ratio)
    expanded_height = height * (1 + expand_ratio)
    
    expanded_x = x - (expanded_width - width) / 2
    expanded_y = y - (expanded_height - height) / 2

    # 截取屏幕区域
    image = ImageGrab.grab(bbox=(expanded_x, expanded_y, expanded_x + expanded_width, expanded_y + expanded_height))
    print('1',expanded_x, expanded_y, expanded_x + expanded_width, expanded_y + expanded_height)
    # 计算实际需要计算颜色平均值的区域坐标
    # 在原始区域内进行平均颜色计算
    crop_x1 = width * (expand_ratio / 2)
    crop_y1 = height * (expand_ratio / 2)
    crop_x2 = crop_x1 + width
    crop_y2 = crop_y1 + height
    
    cropped_image = image.crop((crop_x1, crop_y1, crop_x2, crop_y2))
    print('2',crop_x1, crop_y1, crop_x2, crop_y2)
    # 计算平均颜色
    avg_color = get_average_color(cropped_image)



    
    # # # 计算平均颜色
    # # avg_color = get_average_color(image)
    # avg_color = screen.getpixel((x, y))


    # 将RGB颜色转换为十六进制颜色字符串
    hex_color = '#%02x%02x%02x' % avg_color    
    # 更新Label的背景色
    label.config(bg=hex_color)
    






    
    # 更新
    update_label_background()

def update_data():
    global display_text, heart
    # while not stop_event.is_set():
    #     heart = "1" if heart == "0" else "0"
    #     result = marketDataAPI.get_tickers(instType="SWAP")
    #     result_processes = {
    #         symbol: item['last']
    #         for symbol in instID
    #         for item in result['data']
    #         if f"{symbol}-USDT-SWAP" == item['instId']
    #     }
    #     display_text = "\n".join([f"{symbol:5} : {price:10}" for symbol, price in result_processes.items()])
    #     display_text += f"\nHeart: {heart}"
    #     root.after(0, lambda: label.config(text=display_text))
        # time.sleep(1)

def on_mouse_enter(event):
    global is_dragging
    if not is_dragging:
        root.config(bg='lightgrey')
        # 创建独立的矩形窗口
        create_rectangle_window()

def on_mouse_leave(event):
    global rect_win, canvas
    root.config(bg='white')
    # 删除创建的独立的矩形窗口及其红色边框
    rect_win.destroy()
    # rect_win = None


def on_left_button_down(event):
    global is_dragging, start_x, start_y
    is_dragging = True
    start_x = event.x
    start_y = event.y

def on_left_button_up(event):
    global is_dragging
    is_dragging = False
    check_if_in_rectangle()

def on_mouse_drag(event):
    if is_dragging:
        x = root.winfo_x() + event.x - start_x
        y = root.winfo_y() + event.y - start_y
        root.geometry(f'+{x}+{y}')
        check_if_in_rectangle()

def check_if_in_rectangle():
    win_x = root.winfo_x()
    win_y = root.winfo_y()
    win_width = root.winfo_width()
    win_height = root.winfo_height()

    rect_x = rect_win.winfo_x()
    rect_y = rect_win.winfo_y()
    rect_win_width = rect_win.winfo_width()
    rect_win_height = rect_win.winfo_height()

    if (win_x + win_width > rect_x) and (win_x < rect_x + rect_win_width) and \
       (win_y + win_height > rect_y) and (win_y < rect_y + rect_win_height):
        stop_program()

def stop_program():
    root.destroy()
    rect_win.destroy()
    stop_event.set()
def create_rectangle_window():
    global rect_win, canvas
    if rect_win is not None:
        rect_win.destroy()

    # 获取主窗口的位置和尺寸
    win_x = root.winfo_x()
    win_y = root.winfo_y()
    win_width = root.winfo_width()
    win_height = root.winfo_height()
    
    # 计算矩形窗口的位置
    rect_x = win_x + win_width
    rect_y = win_y

    rect_width = 1000
    rect_height = 600

    rect_win = tk.Toplevel()
    rect_win.geometry(f"{rect_width}x{rect_height}+{int((root.winfo_screenwidth()-rect_width)/5)}+{int((root.winfo_screenheight()-rect_height)/2)}")
    rect_win.overrideredirect(True)
    rect_win.attributes('-topmost', True)
    rect_win.config(bg='yellow')
    rect_win.wm_attributes('-transparentcolor', 'yellow')# 设置背景透明

    # 绘制红色边框
    canvas = tk.Canvas(rect_win, width=rect_width, height=rect_height, highlightthickness=0)
    canvas.pack()
    # canvas.create_rectangle(0, 0, rect_width, rect_height, outline='red', width=10)
    canvas.create_rectangle(0, 0, rect_width, rect_height, outline='red', width=10, tag="one")
    rect_win.lift()

# 绑定鼠标事件
root.bind("<Enter>", on_mouse_enter)
root.bind("<Leave>", on_mouse_leave)
root.bind("<Button-1>", on_left_button_down)
root.bind("<B1-Motion>", on_mouse_drag)
root.bind("<ButtonRelease-1>", on_left_button_up)

# # 启动背景色更新
# update_label_background()

# 启动更新数据的后台线程
update_thread = Thread(target=update_data, daemon=True)
update_thread.start()
update_background = Thread(target=update_label_background, daemon=True)
update_background.start()

# 启动tkinter主循环
root.mainloop()


1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132.0 88.0
1 88.0 912.0 232.0 1008.0
2 12.0 8.0 132

KeyboardInterrupt: 

Exception in thread Thread-6:
Traceback (most recent call last):
  File "e:\conda\envs\py39_tf\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "e:\conda\envs\py39_tf\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\haokw\AppData\Local\Temp\ipykernel_26512\4203808222.py", line 110, in update_label_background
  File "C:\Users\haokw\AppData\Local\Temp\ipykernel_26512\4203808222.py", line 110, in update_label_background
  File "C:\Users\haokw\AppData\Local\Temp\ipykernel_26512\4203808222.py", line 110, in update_label_background
  [Previous line repeated 217 more times]
  File "C:\Users\haokw\AppData\Local\Temp\ipykernel_26512\4203808222.py", line 59, in update_label_background
  File "e:\conda\envs\py39_tf\lib\tkinter\__init__.py", line 1171, in winfo_rootx
    self.tk.call('winfo', 'rootx', self._w))
RuntimeError: main thread is not in main loop


In [ ]:
def create_rectangle_window():
    global rect_win, canvas
    if rect_win is not None:
        rect_win.destroy()


    # 获取主窗口的位置和尺寸
    win_x = root.winfo_x()
    win_y = root.winfo_y()
    win_width = root.winfo_width()
    win_height = root.winfo_height()
    
    # 计算矩形窗口的位置
    rect_x = win_x + win_width
    rect_y = win_y

    rect_width = int(win_width/3)
    rect_height = win_height

    rect_win = tk.Toplevel()
    rect_win.geometry(f"{rect_width}x{rect_height}+{rect_x}+{rect_y}")
    rect_win.overrideredirect(True)
    rect_win.attributes('-topmost', True)
    rect_win.config(bg='yellow')
    rect_win.wm_attributes('-transparentcolor', 'yellow')# 设置背景透明

    # 绘制红色边框
    canvas = tk.Canvas(rect_win, width=rect_width, height=rect_height, highlightthickness=0)
    canvas.pack()
    # canvas.create_rectangle(0, 0, rect_width, rect_height, outline='red', width=10)
    canvas.create_rectangle(0, 0, rect_width, rect_height, outline='red', width=10, fill="red", tag="one")
    rect_win.lift()
